In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("TF-IDF").getOrCreate()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=7a4f1bcf47486a301ab245d02f60c3ed656d771e433adcecf77b8ef2d0b9f67b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [21]:
sc = spark.sparkContext
rdd = sc.textFile("news.txt")


In [22]:
docs_with_index = rdd.zipWithIndex()

tf_with_doc_index = docs_with_index.flatMap(lambda doc: [((word, doc[1]), 1) for word in tokenize(doc[0])])\
                                   .reduceByKey(lambda a, b: a + b)

df = docs_with_index.flatMap(lambda doc: set(tokenize(doc[0])))\
                    .map(lambda word: (word, 1))\
                    .reduceByKey(lambda a, b: a + b)

total_documents = docs_with_index.count()
idf = df.map(lambda x: (x[0], math.log10(total_documents / x[1])))

tf_idf = tf_with_doc_index.map(lambda x: (x[0][0], (x[0][1], x[1])))\
                          .join(idf)\
                          .map(lambda x: (x[0], (x[1][0][0], x[1][0][1] * x[1][1])))\
                          .groupByKey()\
                          .mapValues(list)


In [23]:
def find_related_documents(word, top_n=3):
    return (tf_idf.filter(lambda x: x[0] == word)
                  .flatMap(lambda x: [(doc_id, tfidf) for doc_id, tfidf in x[1]])
                  .takeOrdered(top_n, key=lambda x: -x[1]))

related_documents_japan = find_related_documents("japan")
related_documents_gas = find_related_documents("gas")
related_documents_market = find_related_documents("market")

print("Related documents for 'japan':", related_documents_japan)
print("Related documents for 'gas':", related_documents_gas)
print("Related documents for 'market':", related_documents_market)


Related documents for 'japan': [(6, 4.294091292476962), (5, 1.4313637641589874), (0, 0.9542425094393249)]
Related documents for 'gas': [(0, 1.5563025007672873), (4, 0.7781512503836436)]
Related documents for 'market': [(6, 1.4313637641589874), (7, 0.9542425094393249), (11, 0.9542425094393249)]
